In [1]:
from sqlalchemy import create_engine, text
import numpy as np
import pandas as pd

import credentials

In [191]:
output_file = "helechos_vel_group_aff_2025-11.csv"

In [2]:
conn_str = 'mysql+mysqlconnector://' + \
	f"{credentials.mysql['username']}:{credentials.mysql['password']}" + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

In [3]:
ids = pd.read_sql_table("Identifications", connection)

/Users/nelson/anaconda3/envs/py312/lib/python3.12/site-packages/pandas/io/sql.py:1737: SAWarning: Did not recognize type 'polygon' of column 'Shape'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


In [4]:
noiddate = pd.read_sql(
	"select IdentificationID,DateEnd from Identifications left join Occurrences on Occurrence=OccurrenceID where Date is NULL",
	connection)

In [5]:
ids = ids.merge(noiddate, how='left', on='IdentificationID')

In [6]:
ids.loc[ids.Date.isnull(), 'Date'] = pd.to_datetime(ids.loc[ids.Date.isnull(), 'DateEnd'])

In [ ]:
# Debugging selecting last ided occurrences
#dt = pd.DataFrame.from_dict({
#	'd':['2014-10-16', '2014-10-18', '2014-10-16', '2016-10-16', None, None],
#	'v':[1,1,2,2,2,3],
#	'a':[3,4,4,3,3,4],
#	'i': [1,2,3,4,5,6]})
#dt['d'] = pd.to_datetime(dt.d)
#l = dt.groupby(['v']).d.idxmax()
#onlynas = dt.loc[dt.v.isin(l[l.isnull()].index)].groupby('v').first().reset_index()[dt.columns] # Only NaT dates
#pd.concat([onlynas, dt.loc[l[l.notna()]]])

,d,v,a,i
0,NaT,3,4,6
1,2014-10-18,1,4,2
3,2016-10-16,2,3,4


In [7]:
lastids = ids.groupby('Occurrence').Date.idxmax()

In [8]:
onlynas = ids.loc[
		ids.Occurrence.isin(lastids[lastids.isnull()].index)
	].groupby('Occurrence'
	).first(
	).reset_index(
	)[ids.columns]

In [9]:
id_pool = pd.concat([onlynas, ids.loc[lastids[lastids.notnull()]]]).IdentificationID

In [10]:
tax = pd.read_sql_table("Taxa", connection)

In [11]:
clades = [
	'Polypodiidae',
	'Lycopodiopsida',
	'Ophioglossidae',
	'Equisetidae',
	'Marattiidae'
]

In [12]:
target = []
papa = tax.loc[tax.Name.isin(clades), 'TaxonID'].tolist()

while papa:
	target += papa
	papa = tax.loc[tax.Parent.isin(papa), 'TaxonID'].tolist()
	#print(papa)

In [13]:
pooltax = ', '.join([str(i) for i in target])

In [14]:
tax[tax.TaxonID.isin(target)].head()

,TaxonID,Name,AcceptedName,Authority,Author,Protologue,Parent,Comment,CheckPriority,Distribution,TimeStamp
65,66,Acrostichum aureum,None,None,L.,None,4002.0,None,NaN,None,2025-06-18 09:14:24
66,67,Adiantum,None,None,None,None,2934.0,None,NaN,None,2025-06-18 09:14:24
67,68,Adiantum concinnum,None,None,Humb. & Bonpl. ex Willd.,None,67.0,None,NaN,None,2025-06-18 09:14:24
68,69,Adiantum patens,None,None,Willd.,None,67.0,None,NaN,None,2025-06-18 09:14:24
69,70,Adiantum petiolatum,None,None,Desv.,None,67.0,None,NaN,None,2025-06-18 09:14:24


In [15]:
poollist = ', '.join([str(x) for x in id_pool.tolist()])

In [196]:
recs = pd.read_sql(
	"SELECT Specimens.SpecimenCode AS 'catalogNumber', Occurrences.Type AS 'basisOfRecord', Specimens.Institution AS 'institutionCode', Sources.Author AS 'refAuthor', Sources.Name AS 'refName', Sources.Year AS 'refYear', Sources.Journal AS 'refJournal', Occurrences.CollectorVerbatim AS 'recordedBy', Occurrences.CollectionNumberVerbatim AS 'recordNumber', Occurrences.DateInit AS 'eventDate', Taxa.Name AS 'scientificName', Taxa.Author AS 'scientificNameAuthorship', Identifications.IdentifiedByVerbatim AS 'identifiedBy', Identifications.Date AS 'dateIdentified', Locations.Country AS 'country', Locations.Admin01 AS 'stateProvince', Locations.Admin02 AS 'municipality', Locations.Admin03 as 'county', Locations.Name AS 'localityVerbatim', Geocodings.InterpretedLat AS 'decimalLatitude', Geocodings.InterpretedLon AS 'decimalLongitude', Locations.ElevationMin AS 'minimumElevationInMeters', Locations.ElevationMax AS 'maximumElevationInMeters' FROM Occurrences " \
	+ "LEFT JOIN Sources ON SourceID=Occurrences.Reference " \
	+ "LEFT JOIN Identifications ON Identifications.Occurrence=OccurrenceID " \
	+ "LEFT JOIN Taxa ON TaxonID=Identifications.Name " \
	+ "LEFT JOIN Specimens ON Specimens.Occurrence=OccurrenceID " \
	+ "LEFT JOIN Locations ON LocationID=Occurrences.Location " \
	+ "LEFT JOIN Geocodings ON LocationID=Geocodings.Location " \
	+ f"WHERE Identifications.IdentificationID IN ({poollist}) " \
	+ f"AND Taxa.TaxonID IN ({pooltax})",
	connection)

In [206]:
recs['taxonRank'] = None

In [207]:
recs.loc[recs.scientificName.str.contains(' var. ', regex=False), 'taxonRank'] = 'variety'

In [208]:
recs.loc[recs.scientificName.str.contains(' subsp. ', regex=False), 'taxonRank'] = 'subspecies'

In [209]:
recs.loc[recs.scientificName.str.contains(' fo?\\. ', regex=True), 'taxonRank'] = 'forma'

In [210]:
if recs.loc[recs.scientificName.str.contains('ales$', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('ales$', na=False),
		'taxonRank'] = 'order'

In [211]:
if recs.loc[recs.scientificName.str.contains('aceae$', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('aceae$', na=False),
		'taxonRank'] = 'family'

In [212]:
if recs.loc[recs.scientificName.str.contains('×', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('×', na=False),
		'taxonRank'] = 'hybrid'

In [213]:
recs.loc[
	recs.taxonRank.isnull() &
	recs.scientificName.str.contains('[\\w\\-]\\s+[\\w\\-]', regex=True),
	'taxonRank'] = 'species'

In [214]:
# everything else is ided to genus level
if recs.loc[recs.taxonRank.isnull()].shape[0] > 0:
	recs.loc[recs.taxonRank.isnull(), 'taxonRank'] = 'genus'

In [216]:
recs['genus'] = None
recs['specificEpithet'] = None
recs['infraspecificEpithet'] = None

In [219]:
recs.loc[recs.taxonRank == 'species', 'specificEpithet'
	] = recs.loc[recs.taxonRank == 'species', 'scientificName'
	].str.extract('\\s+([\\w\\-]+)$')[0]

In [220]:
recs.loc[recs.taxonRank.isin(['variety', 'forma', 'subspecies']), 'specificEpithet'
	] = recs.loc[recs.taxonRank.isin(['variety', 'forma', 'subspecies']), 'scientificName'
	].str.extract('\\s+([\\w\\-]+)\\s+[\\w\\-]+$')

In [221]:
recs.loc[recs.taxonRank.isin(['variety', 'forma', 'subspecies']), 'infraspecificEpithet'
	] = recs.loc[recs.taxonRank.isin(['variety', 'forma', 'subspecies']), 'scientificName'
	].str.extract('\\s+[\\w\\-]+\\s+([\\w\\-]+)$')

In [222]:
recs.loc[recs.taxonRank == 'genus', 'genus'
	] = recs.loc[recs.taxonRank == 'genus', 'scientificName']

In [224]:
recs.loc[
	recs.taxonRank.isin(['species', 'hybrid', 'variety', 'forma', 'subspecies']),
	'genus'
	] = recs.loc[
	recs.taxonRank.isin(['species', 'hybrid', 'variety', 'forma', 'subspecies'])
	].scientificName.str.replace('×', ' '
	).str.replace('^\\s+|\\s+$', '', regex=True
	).str.replace('\\s+', ' ', regex=True
	).str.split('\\s+', expand=True
	)[0]

In [225]:
def get_family(row):
	if row.taxonRank == 'family':
		return row.scientificName
	elif row.genus:
		thpapa = tax.loc[tax.Name == row.genus, 'Parent'].item()
		return tax.loc[tax.TaxonID == thpapa, 'Name'].item()
	else:
		return None


In [226]:
recs['family'] = recs.apply(get_family, axis=1)

In [227]:
recs.columns

Index(['catalogNumber', 'basisOfRecord', 'institutionCode', 'refAuthor',
       'refName', 'refYear', 'refJournal', 'recordedBy', 'recordNumber',
       'eventDate', 'scientificName', 'scientificNameAuthorship',
       'identifiedBy', 'dateIdentified', 'country', 'stateProvince',
       'municipality', 'county', 'localityVerbatim', 'decimalLatitude',
       'decimalLongitude', 'minimumElevationInMeters',
       'maximumElevationInMeters', 'taxonRank', 'genus', 'specificEpithet',
       'infraspecificEpithet', 'family'],
      dtype='object')

In [228]:
recs = recs[
	['catalogNumber', 'basisOfRecord', 'institutionCode',
	'refAuthor', 'refName', 'refYear', 'refJournal',
	'recordedBy', 'recordNumber', 'eventDate', 
	'taxonRank', 'scientificName', 'scientificNameAuthorship', 
	'family', 'genus', 'specificEpithet', 'infraspecificEpithet',
	'identifiedBy', 'dateIdentified', 
	'country', 'stateProvince', 'municipality', 'county', 'localityVerbatim',
	'decimalLatitude', 'decimalLongitude', 
	'minimumElevationInMeters', 'maximumElevationInMeters']
]

In [229]:
recs.to_csv(output_file, index=False)

In [55]:
connection.close()